In [1]:
import os
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.optimizers import Adam


In [2]:
# File path to JSON dataset
file_path = r'C:\Users\Administrator\Documents\GitHub\Datasets\yelp_academic_dataset_review.json'

# Load data in chunks
chunk_size = 1000000
model_save_path = 'sentiment_analysis_cnn_model.h5'

# Initialize the model outside the loop
model = None
epochs = 10


In [3]:
for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
    print(chunk)
    # Accessing the data DataFrame
    texts = chunk['text'].tolist()
    labels = chunk['stars'].tolist()

    # Tokenize the text
    tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    # Padding sequences
    max_len = 100 
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    # Convert labels to numerical format
    numeric_labels = np.array(labels)

    # Split the dataset into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(padded_sequences, numeric_labels, test_size=0.2, random_state=42)

    # Build or load the CNN model
    if model is None or not os.path.exists(model_save_path):
        embedding_dim = 50
        filters = 64
        kernel_size = 3

        model = Sequential()
        model.add(Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_len))
        model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))
        model.add(GlobalMaxPooling1D())
        model.add(Dense(1, activation='linear'))  # Used linear activation for regression
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')  # Use mean squared error for regression
    else:
        # Load the previously trained model
        model = load_model(model_save_path)

    # Train the model
    model.fit(x_train, y_train, epochs=epochs, batch_size=128, validation_split=0.1)


                     review_id                 user_id  \
0       KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA   
1       BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q   
2       saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A   
3       AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ   
4       Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ   
...                        ...                     ...   
999995  t-2o35kr7Q9DSaeuKhaDuQ  oX7o1TH0PHUWp9r9ry9_vw   
999996  fLIwWCvdul9PNWYfJt5QWA  v8wlapFKVLs2qTYCGhCdiw   
999997  ETAiy6wEM-r9ve4SKDhBpg  rLlYc1RzIBnOmnX3AbpEYw   
999998  8OgvSXuc6KjAt2fSz9LuzA  eEH-8CEPU5ndPxDGzVfHiQ   
999999  TQoVzNXqDkiDXgzlw8cyaQ  HYmGwYXvcYmW7dDjuWKJfw   

                   business_id  stars  useful  funny  cool  \
0       XQfwVwDr-v0ZS3_CbbE5Xw      3       0      0     0   
1       7ATYjTIgM3jUlt4UM3IypQ      5       1      0     1   
2       YjUWPpI6HXG530lwP-fb2A      3       0      0     0   
3       kxX2SOes4o-D3ZQBkiMRfA      5       1      0   

Epoch 1/10
5625/5625 [==============================] - 29s 5ms/step - loss: 0.7525 - val_loss: 0.5665
Epoch 2/10
5625/5625 [==============================] - 28s 5ms/step - loss: 0.5176 - val_loss: 0.5421
Epoch 3/10
5625/5625 [==============================] - 29s 5ms/step - loss: 0.4725 - val_loss: 0.5429
Epoch 4/10
5625/5625 [==============================] - 26s 5ms/step - loss: 0.4397 - val_loss: 0.5479
Epoch 5/10
5625/5625 [==============================] - 27s 5ms/step - loss: 0.4120 - val_loss: 0.5531
Epoch 6/10
5625/5625 [==============================] - 28s 5ms/step - loss: 0.3898 - val_loss: 0.5631
Epoch 7/10
5625/5625 [==============================] - 28s 5ms/step - loss: 0.3704 - val_loss: 0.5731
Epoch 8/10
5625/5625 [==============================] - 28s 5ms/step - loss: 0.3539 - val_loss: 0.5859
Epoch 9/10
5625/5625 [==============================] - 28s 5ms/step - loss: 0.3393 - val_loss: 0.5946
Epoch 10/10
5625/5625 [==============================] - 28s 5ms/step - l

Epoch 1/10
5625/5625 [==============================] - 55s 10ms/step - loss: 0.7307 - val_loss: 0.5451
Epoch 2/10
5625/5625 [==============================] - 54s 10ms/step - loss: 0.5091 - val_loss: 0.5276
Epoch 3/10
5625/5625 [==============================] - 54s 10ms/step - loss: 0.4655 - val_loss: 0.5284
Epoch 4/10
5625/5625 [==============================] - 54s 10ms/step - loss: 0.4332 - val_loss: 0.5319
Epoch 5/10
5625/5625 [==============================] - 54s 10ms/step - loss: 0.4061 - val_loss: 0.5376
Epoch 6/10
5625/5625 [==============================] - 54s 10ms/step - loss: 0.3840 - val_loss: 0.5465
Epoch 7/10
5625/5625 [==============================] - 54s 10ms/step - loss: 0.3646 - val_loss: 0.5555
Epoch 8/10
5625/5625 [==============================] - 54s 10ms/step - loss: 0.3482 - val_loss: 0.5647
Epoch 9/10
5625/5625 [==============================] - 54s 10ms/step - loss: 0.3336 - val_loss: 0.5786
Epoch 10/10
5625/5625 [==============================] - 54s 10m

In [5]:
# Save the entire model to an HDF5 file
model.save('final_cnn_model.h5')